In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
park = gpd.read_file('./geo_files/3_3_parking_space.geojson')

In [3]:
park.columns

Index(['OBJECTID', 'BLOCKID', 'ROWNBR', 'BLOCK_ST', 'BLOCK_END', 'WIDTH_OFFS',
       'GEOBASYS', 'SPACELENGT', 'SPACETYPE', 'SPACETYPED', 'TIME_LIMIT',
       'SPACE_NB', 'CATEGORY', 'SIDE', 'CURRENT_ST', 'ELMNTKEY', 'SHAPE_Leng',
       'geometry'],
      dtype='object')

In [4]:
park.SPACETYPE.unique()

array(['ALLEY', 'PS', 'CLR', 'PLZ', 'PS-CAR', 'CRBBLB', 'NP', 'CVLZ',
       'BA', 'XWAREA', 'XW', 'HYD', 'CR', 'TAZ', 'PS-PLZ', 'BUS',
       'PS-TAX', 'BIKE', 'PS-LAY', 'PS-MC', 'DW', 'CVLZ-B', 'CVLZ-P',
       'PS-SBO', 'PARKLET', 'L/UL', 'BUSLAY', 'PS-TR'], dtype=object)

In [5]:
park.SPACETYPED.unique()

array(['ALLEY', 'PAY STATION', 'CLEARANCE', 'PASSENGER LOAD ZONE',
       'PAY STATION - CARPOOL', 'CURB BULB', 'NO PARKING',
       'COMMERCIAL VEHICLE LOAD ZONE', 'BARRIER AREA',
       'NOT A DESIGNATED CROSSWALK', 'CROSSWALK', 'HYDRANT', 'CURB RAMP',
       'TOW AWAY ZONE', 'PAY STATION - PLZ', 'BUS ZONE',
       'PAY STATION - TAXI', 'BIKE ZONE', 'PAY STATION - BUS LAYOVER',
       'PAY STATION - MOTORCYCLE', 'DRIVEWAY', 'CVLZ - BUS ZONE',
       'CVLZ - PLZ', 'PAY STATION - SHUTTLE BUS ONLY', 'PARKLET',
       'LOAD/UNLOAD ZONE', 'BUS LAYOVER ZONE',
       'PAY STATION - TEMP REMOVAL'], dtype=object)

In [6]:
# select parking space types
# original: PLZ, L/UL, CVLZ, PS and PS-X
park = park[park['SPACETYPE'].isin(['PLZ', 'L/UL',
                                  'CVLZ', 'CVLZ-B', 'CVLZ-P',
                                  'PS', 'PS-CAR', 'PS_PLZ', 'PS-TAX', 'PS-LAY', 'PS-SBO'])]

In [7]:
# filter out links that are not in the 3*3 blocks
park = park[park['BLOCKID'].apply(lambda x: x.endswith(tuple(['24','25','26','01','02','03'])))]

In [8]:
park.replace(to_replace=r'^PS-\w+', value='PS', regex=True, inplace=True)

In [9]:
# count and length of parking
pc = park.groupby(['BLOCKID', 'SIDE', 'SPACETYPE']).agg({'SPACELENGT': 'sum', 'OBJECTID': 'count'}).reset_index()
# pc.rename(columns={'SPACELENGT': 'length', 0:'cnt'}, inplace=True)

In [10]:
pc

,BLOCKID,SIDE,SPACETYPE,SPACELENGT,OBJECTID
0,01-24,NE,CVLZ,23,1
1,01-24,NE,PLZ,22,1
2,01-24,NE,PS,143,8
3,01-24,SW,PLZ,38,2
4,01-24,SW,PS,58,3
...,...,...,...,...,...
75,27-02,SE,PS,88,5
76,27-03,NW,CVLZ,18,1
77,27-03,NW,PLZ,18,1
78,27-03,NW,PS,119,7


In [11]:
pc.columns = ['block_id', 'side', 'type', 'length', 'cnt']

In [12]:
pc.head()

,block_id,side,type,length,cnt
0,01-24,NE,CVLZ,23,1
1,01-24,NE,PLZ,22,1
2,01-24,NE,PS,143,8
3,01-24,SW,PLZ,38,2
4,01-24,SW,PS,58,3


In [13]:
pc.to_csv('parking_space_count.csv')

In [14]:
pc.type.unique()

array(['CVLZ', 'PLZ', 'PS', 'CVLZ-P', 'CVLZ-B', 'L/UL'], dtype=object)

In [16]:
park[park['BLOCKID'] == '01-24']

,OBJECTID,BLOCKID,ROWNBR,BLOCK_ST,BLOCK_END,WIDTH_OFFS,GEOBASYS,SPACELENGT,SPACETYPE,SPACETYPED,TIME_LIMIT,SPACE_NB,CATEGORY,SIDE,CURRENT_ST,ELMNTKEY,SHAPE_Leng,geometry
21,590,01-24,None,260.0,277.0,-27,None,17,PS,PAY STATION,240,-20,PAID,NE,INSVC,1026,16.982663,"MULTILINESTRING ((-122.34853 47.61467, -122.34..."
34,868,01-24,None,101.5,121.5,30,None,20,PS,PAY STATION,240,-07,PAID,SW,INSVC,1025,19.979641,"MULTILINESTRING ((-122.34820 47.61427, -122.34..."
61,1638,01-24,None,52.5,70.5,-27,None,18,PS,PAY STATION,240,-02,PAID,NE,INSVC,1026,17.981685,"MULTILINESTRING ((-122.34790 47.61429, -122.34..."
72,1870,01-24,None,121.0,144.0,-27,None,23,CVLZ,COMMERCIAL VEHICLE LOAD ZONE,30,-08T,LOAD,NE,INSVC,1026,22.976572,"MULTILINESTRING ((-122.34811 47.61442, -122.34..."
74,1917,01-24,None,56.0,75.0,30,None,19,PLZ,PASSENGER LOAD ZONE,3,-03,ZONE,SW,INSVC,1025,18.980620,"MULTILINESTRING ((-122.34806 47.61418, -122.34..."
80,2098,01-24,None,75.0,94.0,30,None,19,PS,PAY STATION,240,-05,PAID,SW,INSVC,1025,18.980663,"MULTILINESTRING ((-122.34812 47.61422, -122.34..."
88,2264,01-24,None,78.0,96.0,-27,None,18,PS,PAY STATION,240,-04,PAID,NE,INSVC,1026,17.981689,"MULTILINESTRING ((-122.34798 47.61434, -122.34..."
95,2459,01-24,None,96.0,114.0,-27,None,18,PS,PAY STATION,240,-06,PAID,NE,INSVC,1026,17.981685,"MULTILINESTRING ((-122.34803 47.61437, -122.34..."
110,2850,01-24,None,174.0,192.0,-27,None,18,PS,PAY STATION,240,-12,PAID,NE,INSVC,1026,17.981641,"MULTILINESTRING ((-122.34827 47.61451, -122.34..."
112,2898,01-24,None,121.5,140.5,30,None,19,PS,PAY STATION,240,-09,PAID,SW,INSVC,1025,18.980620,"MULTILINESTRING ((-122.34826 47.61430, -122.34..."
